***  Time Series Forecast  ***

***
Methods
Data set is split to train and test subsets, applied objects and functions:

*   Loading and cleaning data frame using pandas and Python’s datetime class
*   statsmodels.tsa class
*   numpy
*   statsmodels.tsa
*   Model evaluation using sklearn.metrics (RMSE — root mean squared error), and comparison of observed and forecast value integrals 

**STEPS**


1.   **Install and import packages**
2.   **Import data**
3.   **Forecast by using whole data**
4.   **Compare performance and conclusion**


***Import the packages***

In [ ]:
pip install pmdarima

In [ ]:
pip install autots

In [ ]:
pip install auto_ts

In [ ]:
pip install "dask[complete]"

In [ ]:
#mathematical operations
import numpy as np
# from numpy.random import seed
# from numpy.random import normal
import math
import scipy as sp
from scipy.stats import norm

#data handling
import pandas as pd
from pandas import read_csv
import pandas.util.testing as tm
from sklearn.model_selection import train_test_split

# visualization plotting
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns             # statistical data visualization
sns.set()
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline

#dataframe index manipulations
from datetime import datetime

#muting unnecessary warnings if needed
import warnings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  if sys.path[0] == '':


In [ ]:
# machine learning and statistical methods
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose # Error Trend Seasonality decomposition
from statsmodels.tsa.filters.hp_filter import hpfilter  # Hodrick Prescott filter for cyclic & trend separation
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   # single exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES # double and triple exponential smoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import Holt as Holt
#from statsmodels.tsa.arima.model import ARIMA

import pmdarima as pm
from pmdarima.arima import auto_arima
from fbprophet import Prophet
from autots import AutoTS
from auto_ts import auto_timeseries as ATS
import xgboost

from sklearn.model_selection import TimeSeriesSplit as tscv

Imported auto_timeseries version:0.0.64. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



In [ ]:
#selected preprocessing and evaluation methods
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from scipy.stats import shapiro

***Import Data***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading raw data
path = "/content/drive/MyDrive/IS/code/Data1.csv"
df=pd.read_csv(path)
df=df[['date', 'title','quantity']]
df=df.rename(columns= {'title':'menu'})
df.head()

,date,menu,quantity
0,2021-02-17,หมูเด้งต้มยำ,1.0
1,2021-02-17,หมูเด้งต้มยำ,1.0
2,2021-02-17,หมูเด้งต้มยำ,1.0
3,2021-02-17,หมูเด้งต้มยำ,1.0
4,2021-02-17,หมูเด้งต้มยำ,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149554 entries, 0 to 149553
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      149554 non-null  object 
 1   menu      149554 non-null  object 
 2   quantity  149550 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.4+ MB


In [ ]:
# Find unique menu
n_menu =df['menu'].nunique()
print(f'menu: ', n_menu)
print(f'-------------------------------------------------------------------------')
menu = df['menu'].unique()
print(menu)

menu:  23
-------------------------------------------------------------------------
['หมูเด้งต้มยำ' 'ก๋วยเตี๋ยวต้มยำเครื่องทะลัก' 'ข้าวซอยไก่' 'เล้งแซ่บ'
 'ขนมจีนน้ำเงี้ยว' 'ต้มเลือดหมู' 'หมูต้มยำ' 'ต้มยำทะเล'
 'ข้าวผัดพริกเผาทะเล' 'ข้าวผัดหมู' 'ยำวุ้นเส้นทะเล' 'ยำเกี๊ยวกรอบ'
 'หมูต้มยำขลุกขลิก' 'ต้มยำกุ้ง' 'ยำขนมจีนปลาทู' 'หมูจัมโบ้'
 'ต้มยำหมึกยัดไส้' 'ต้มยำหมึก' 'ข้าวกะเพราหมู' 'ข้าวหมูผัดพริกแกงใต้'
 'ต้มยำกุ้งแม่น้ำ' 'ข้าวน้ำพริกปลาทู' 'ลวกจิ้มหมู']


In [ ]:
type(menu)

numpy.ndarray

In [ ]:
df = df.groupby(['date', 'menu'])['quantity'].agg("sum").reset_index()

print(df)

             date              menu  quantity
0      2019-07-04   ขนมจีนน้ำเงี้ยว       1.0
1      2019-07-04     ข้าวกะเพราหมู       2.0
2      2019-07-04        ข้าวซอยไก่       5.0
3      2019-07-04        ข้าวผัดหมู       2.0
4      2019-07-04         ต้มยำทะเล       1.0
...           ...               ...       ...
16902  2022-01-12         หมูจัมโบ้       1.0
16903  2022-01-12          หมูต้มยำ      24.0
16904  2022-01-12  หมูต้มยำขลุกขลิก       3.0
16905  2022-01-12      หมูเด้งต้มยำ      26.0
16906  2022-01-12          เล้งแซ่บ       2.0

[16907 rows x 3 columns]


In [ ]:
#converting dataframe indices to datetime
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.dayofyear
df['weekday'] = df['date'].dt.weekday

df.head()

,date,menu,quantity,year,month,day,weekday
0,2019-07-04,ขนมจีนน้ำเงี้ยว,1.0,2019,7,185,3
1,2019-07-04,ข้าวกะเพราหมู,2.0,2019,7,185,3
2,2019-07-04,ข้าวซอยไก่,5.0,2019,7,185,3
3,2019-07-04,ข้าวผัดหมู,2.0,2019,7,185,3
4,2019-07-04,ต้มยำทะเล,1.0,2019,7,185,3


In [ ]:
# sales = df.loc[(df['date'] >= '2021-12-14')]
# sales = sales[['date', 'menu','quantity']]
# sales

In [ ]:
# import openpyxl
# sales.to_excel('Sales_210425.xlsx', sheet_name='Sales30')

In [ ]:
df['menu'].isnull().sum()

0

In [ ]:
# Try on this!!! (last 30)
def forecast(train_df, test_df, test_size):
  #SES
  model_ses = SimpleExpSmoothing(train_df.quantity).fit() #smoothing_level = alpha, optimized = False)
  ses = model_ses.forecast(test_size)
  mean_s = ses.to_frame().iloc[-30:]
  mean_s = mean_s.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_s = pd.DataFrame(mean_s, columns= ['demand']).astype(int)
  mean_s = mean_s.transpose()
  # Calculate RMSE
  rmse_s = mean_squared_error(test_df['quantity'], ses, squared=False)
  resultS = pd.DataFrame({'Method':'ses','RMSE': ['%.3f' %(rmse_s)]})
  resultS.reset_index(drop=True, inplace=True)
  mean_s.reset_index(drop=True, inplace=True)
  result_ses = pd.concat([resultS, mean_s], axis=1)

  #holt
  model_h = Holt(train_df.quantity)
  fit_h = model_h.fit(optimized=True) # fit_h = model.fit(smoothing_level=.3, smoothing_slope=.05)
  holt = fit_h.forecast(test_size)
  mean_h = holt.to_frame().iloc[-30 :]
  mean_h = mean_h.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_h = pd.DataFrame(mean_h, columns= ['demand']).astype(int)
  mean_h = mean_h.transpose()
  # Calculate RMSE
  rmse_h = mean_squared_error(test_df['quantity'], holt, squared=False)
  resultH = pd.DataFrame({'Method':'holt','RMSE': ['%.3f' %(rmse_h)]})
  resultH.reset_index(drop=True, inplace=True)
  mean_h.reset_index(drop=True, inplace=True)
  result_h = pd.concat([resultH, mean_h], axis=1)
  
  #holt_damp
  model_hd = Holt(train_df.quantity, damped_trend=True, initialization_method="estimated").fit()
  holt_damp = model_hd.forecast(test_size)

  mean_hd = holt_damp.to_frame().iloc[-30 :]
  mean_hd = mean_hd.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_hd = pd.DataFrame(mean_hd, columns= ['demand']).astype(int)
  mean_hd = mean_hd.transpose()
  # Calculate RMSE
  rmse_hd = mean_squared_error(test_df['quantity'], holt_damp, squared=False)
  resultHd = pd.DataFrame({'Method':'holt_damp','RMSE': ['%.3f' %(rmse_hd)]})
  resultHd.reset_index(drop=True, inplace=True)
  mean_hd.reset_index(drop=True, inplace=True)
  result_hd = pd.concat([resultHd, mean_hd], axis=1)

  #holtwinter_additive
  model_hw_ad = HWES(train_df ['quantity'],seasonal='add',seasonal_periods=7).fit()
  holtwinter_additive = model_hw_ad.forecast(test_size)
  
  mean_hw_ad = holtwinter_additive.to_frame().iloc[-30 :]
  mean_hw_ad = mean_hw_ad.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_hw_ad = pd.DataFrame(mean_hw_ad, columns= ['demand']).astype(int)
  mean_hw_ad = mean_hw_ad.transpose()
  # Calculate RMSE
  rmse_hw_ad = mean_squared_error(test_df['quantity'], holtwinter_additive, squared=False)
  resultHA = pd.DataFrame({'Method':'holtwinter_additive','RMSE': ['%.3f' %(rmse_hw_ad)]})
  resultHA.reset_index(drop=True, inplace=True)
  mean_hw_ad.reset_index(drop=True, inplace=True)
  result_hw_ad = pd.concat([resultHA, mean_hw_ad], axis=1)


  #holtwinter_multiplicative
  model_hw_m = HWES(train_df ['quantity'],seasonal='mul',seasonal_periods=7).fit()
  holtwinter_multiplicative = model_hw_m.forecast(test_size)
  mean_hw_m = holtwinter_multiplicative.to_frame().iloc[-30 :]
  mean_hw_m = mean_hw_m.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_hw_m = pd.DataFrame(mean_hw_m, columns= ['demand']).astype(int)
  mean_hw_m = mean_hw_m.transpose()
  # Calculate RMSE
  rmse_hw_m = mean_squared_error(test_df['quantity'], holtwinter_multiplicative, squared=False)
  resultHM = pd.DataFrame({'Method':'holtwinter_multiplicative','RMSE': ['%.3f' %(rmse_hw_m)]})
  resultHM.reset_index(drop=True, inplace=True)
  mean_hw_m.reset_index(drop=True, inplace=True)
  result_hw_m = pd.concat([resultHM, mean_hw_m], axis=1)


  #holtwinter_multiplicative_damp
  model_hw_md = HWES(train_df.quantity,trend='mul', seasonal='mul', damped =True,seasonal_periods=7).fit() #seasonal=None
  holtwinter_multiplicative_damp = model_hw_md.forecast(test_size)
  mean_hm = holtwinter_multiplicative_damp.to_frame().iloc[-30 :]
  mean_hm = mean_hm.reset_index().rename(columns={'index':'date', 0 :'demand'}) 
  mean_hm = pd.DataFrame(mean_hm, columns= ['demand']).astype(int)
  mean_hm = mean_hm.transpose()
  # Calculate RMSE
  rmse_hm = mean_squared_error(test_df['quantity'], holtwinter_multiplicative_damp, squared=False)
  resultHMd = pd.DataFrame({'Method':'holtwinter_multiplicative_damp','RMSE': ['%.3f' %(rmse_hm)]})
  resultHMd.reset_index(drop=True, inplace=True)
  mean_hm.reset_index(drop=True, inplace=True)
  result_hw_md = pd.concat([resultHMd, mean_hm], axis=1)

  #fbprophet
  m = Prophet(daily_seasonality=True, seasonality_mode='additive')
  alldf = train_df.rename(columns={'date':'ds', 'quantity':'y'})
  m.fit(alldf)
  future = m.make_future_dataframe(periods=1 ,freq='D') 
  prophet_pred = m.predict(future)
  prophet_pred = prophet_pred[['ds','yhat']]
  fbprophet=prophet_pred.round(0)
  fbprophet= fbprophet.rename(columns={'ds':'date', 'yhat':'quantity'})
  fbprophet['date'] = pd.to_datetime(fbprophet.date, format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
  fbprophet = fbprophet.iloc[-test_size:]
  fb = fbprophet.tail(30)
  mean_f = fb['quantity'] 
  mean_f = pd.DataFrame(mean_f, columns= ['quantity']).astype(int)
  mean_f = mean_f.transpose()
  # Calculate RMSE
  rmse_f = mean_squared_error(test_df['quantity'], fbprophet['quantity'], squared=False) 
  resultF = pd.DataFrame({'Method':'fbprophet','RMSE': ['%.3f' %(rmse_f)]}) 
  resultF.reset_index(drop=True, inplace=True)
  mean_f.reset_index(drop=True, inplace=True)
  result_fb = pd.concat([resultF, mean_f], axis=1)

  #arima
  model_arima = auto_arima(train_df['quantity'], trace=True, error_action='ignore', suppress_warnings=True)
  model_arima.fit(train_df['quantity'])
  forecast_arima = model_arima.predict(n_periods=test_size)
  arima = pd.DataFrame(forecast_arima,index = test_df['quantity'].index,columns=['quantity'])
  a = arima.iloc[-30:]
  mean_a = a['quantity'] 
  mean_a = pd.DataFrame(mean_a, columns= ['quantity']).astype(int)
  mean_a = mean_a.transpose()
  # Calculate RMSE
  rmse_a = mean_squared_error(test_df['quantity'], arima, squared=False) 
  resultA = pd.DataFrame({'Method':'arima','RMSE': ['%.3f' %(rmse_a)]}) 
  resultA.reset_index(drop=True, inplace=True)
  mean_a.reset_index(drop=True, inplace=True)
  result_A = pd.concat([resultA, mean_f], axis=1)
  
  #XGB
  X = train_df.iloc[:,1:-1].values
  y = train_df.iloc[:, -1].values
  # Create train & test datasets
  #from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = \
          train_test_split(X, y, test_size = 0.12, \
                          random_state = 0, shuffle=False)
  # Scale the independent variables
  from sklearn.preprocessing import StandardScaler
  scalerX = StandardScaler().fit(X_train)
  X_train_scaled = scalerX.transform(X_train)
  X_test_scaled = scalerX.transform(X_test)
  X_test_df = test_df.iloc[:,1:-1].values
  X_test_df_scaled = scalerX.transform(X_test_df)
  reg = xgboost.XGBRegressor(objective='reg:squarederror', \
                            n_estimators=1000, \
                            nthread=24)
  reg.fit(X_train_scaled, y_train)
  predict_xgb = reg.predict(X_test_df)
  predict_xgb = pd.DataFrame({'Predictions': \
                                      predict_xgb})
  result_xgb = pd.concat( \
                        [df.tail(len(X_test_df)) \
                                .reset_index(drop=True), \
                          predict_xgb], axis=1)

  xgb = result_xgb[['date', 'Predictions']]
  mean_x = xgb.iloc[-30:]
  mean_x = mean_x ['Predictions'] 
  mean_x = pd.DataFrame(mean_x, columns= ['Predictions']).astype(int)
  mean_x = mean_x.transpose()
  # Calculate RMSE
  rmse_x = mean_squared_error(test_df['quantity'], xgb['Predictions'], squared=False) 
  resultX = pd.DataFrame({'Method':'xgb','RMSE': ['%.3f' %(rmse_x)]}) 
  resultX.reset_index(drop=True, inplace=True)
  mean_x.reset_index(drop=True, inplace=True)
  result_X = pd.concat([resultX, mean_x], axis=1)

  #compare model
  result_ses.reset_index(drop=True, inplace=True)
  result_h.reset_index(drop=True, inplace=True)
  result_hd.reset_index(drop=True, inplace=True)
  result_hw_ad.reset_index(drop=True, inplace=True)
  result_hw_m.reset_index(drop=True, inplace=True)
  result_hw_md.reset_index(drop=True, inplace=True)
  result_fb.reset_index(drop=True, inplace=True)
  result_A.reset_index(drop=True, inplace=True)
  result_X.reset_index(drop=True, inplace=True)
  bestDf = pd.DataFrame(np.concatenate([result_ses.values, result_h.values, \
                                        result_hd.values, result_hw_ad.values, \
                                        result_hw_m.values, result_hw_md.values, \
                                        result_fb.values, result_A.values, \
                                        result_X.values]), columns = result_ses.columns)   #axis=0,   
  
  bestDf['RMSE'] = bestDf['RMSE'].astype('float64')
  bestDf = bestDf.sort_values('RMSE').drop_duplicates(['RMSE'], keep='first')
  best = bestDf[bestDf['RMSE'] == bestDf['RMSE'].min()]
  

  #Calculate S.D.
  sdv = train_df.quantity.std()
  sdvDf = pd.DataFrame({'S.D.': ['%.2f' %(sdv)]})

  best.reset_index(drop=True, inplace=True)
  sdvDf.reset_index(drop=True, inplace=True)
  best = pd.concat([best, sdvDf], axis=1)

  return best

In [ ]:
AllBestModel = pd.DataFrame(columns = ['Menu', 'Method','RMSE','1','2','3','4','5','6','7','8','9','10',\
                                       '11','12','13','14','15','16','17','18','19','20',\
                                       '21','22','23','24','25','26','27','28','29', '30','S.D.'])

In [ ]:
nub = 0

In [ ]:
# Try for loop HERE ;w;
for m in menu:
  for n in df['menu']:
    if (n == m):
      eiei = df[(df['menu']  == n)]
      #print(eiei)

  eiei.drop(['menu'], axis=1, inplace=True)
      
  #Count data
  data =eiei['date'].nunique()
  #Train/Test Split                                
  split = round(data * 0.70) 
  train_df = eiei[:split] 
  test_df = eiei[split:]
  test_size = len(test_df)
  

  bestModel = forecast(train_df, test_df, test_size)
  print("Best Model",bestModel)

  new_row = {'Menu': m, 'Method':bestModel['Method'].item(), 'RMSE':bestModel['RMSE'].item(), \
             '1':bestModel[0].item(), '2':bestModel[1].item(), '3':bestModel[2].item(), '4':bestModel[3].item(),\
             '5':bestModel[4].item(), '6':bestModel[5].item(), '7':bestModel[6].item(), '8':bestModel[7].item(),\
             '9':bestModel[8].item(), '10':bestModel[9].item(), '11':bestModel[10].item(), '12':bestModel[11].item(),\
             '13':bestModel[12].item(), '14':bestModel[13].item(), '15':bestModel[14].item(), '16':bestModel[15].item(),\
             '17':bestModel[16].item(), '18':bestModel[17].item(), '19':bestModel[18].item(), '20':bestModel[19].item(),\
             '21':bestModel[20].item(), '22':bestModel[21].item(), '23':bestModel[22].item(), '24':bestModel[23].item(),\
             '25':bestModel[24].item(), '26':bestModel[25].item(), '27':bestModel[26].item(), '28':bestModel[27].item(),\
             '29':bestModel[28].item(), '30':bestModel[29].item(), 'S.D.':bestModel['S.D.'].item()}  # 
  
 
  AllBestModel = AllBestModel.append(new_row, ignore_index = True)
  print("Menu Number: ", nub)
  nub += 1


print("I'm DONE!!!!!!")

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=4856.817, Time=0.53 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=5150.699, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=5013.975, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4858.805, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=5148.703, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=4859.295, Time=0.36 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=4855.486, Time=0.32 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=4860.797, Time=0.21 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=4915.603, Time=0.18 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=4857.351, Time=0.42 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=4892.360, Time=0.23 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=4858.595, Time=1.01 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=4853.587, Time=0.29 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=4858.927, Time=0.22 sec
 ARIMA(2,1,0)(0,0,0)[0]          

In [ ]:
AllBestModel 

,Menu,Method,RMSE,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,S.D.
0,หมูเด้งต้มยำ,arima,9.591,47,54,49,46,48,45,46,...,47,54,49,46,48,45,46,47,53,19.81
1,ก๋วยเตี๋ยวต้มยำเครื่องทะลัก,holt,0.561,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0.84
2,ข้าวซอยไก่,fbprophet,6.173,17,17,18,16,17,18,21,...,17,17,18,16,17,18,21,17,18,7.25
3,เล้งแซ่บ,holtwinter_multiplicative_damp,1.754,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,3.22
4,ขนมจีนน้ำเงี้ยว,holt_damp,6.403,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,12.45
5,ต้มเลือดหมู,fbprophet,4.463,10,11,11,11,10,10,10,...,10,11,11,11,10,11,10,10,11,4.80
6,หมูต้มยำ,fbprophet,10.196,23,25,20,20,20,19,19,...,22,24,18,19,18,18,18,21,24,15.60
7,ต้มยำทะเล,xgb,4.456,3,3,3,3,3,3,3,...,3,3,2,2,3,3,3,3,3,9.22
8,ข้าวผัดพริกเผาทะเล,ses,2.133,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,2.89
9,ข้าวผัดหมู,holt,4.513,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,8.55


In [ ]:
import openpyxl
AllBestModel.to_excel('FC_AllMenu210425.xlsx', sheet_name='My_Best')

----------------------------------------------------------------------------